Downloads description of articles
Cargo un archivo en forma de dataframe, exitraigo un sub-dataframe, scrapeo el comentario de wallapop y lo retorno en un df que es id - descripcion

In [20]:
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
# Función de scrapeo comentarios
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

In [21]:
# Selecciono el CSV
elemento_a_buscar = "iphone" # para guardar el archivo   <--------
#iphone = int((elemento_a_buscar).split()[1]) # 12, 14 o 15

In [22]:
# Cargo archivo
fecha = datetime.now().strftime("%Y%m%d")
#fecha = "20250117"
product = 'iphone'
carpeta_origen = "../data/3_feature_engineering"
carpeta_destino = "../data/4_download_description"

In [23]:
# 263 anuncios - 25 min. con sleep de 1
# 5s por anuncio si va lento. Si va bien 1.5s
# Escrapeo objetos de:
# Vendedor, cuadro datos, cuadro envio, cuadro comentario, cuadro 'editado views y likes'
# por ahora solo comentario

def scrap_comments(df):
    """
    Dado un df con una columna 'url_articulo de wallapop':
    accede a la url_articulo, 
    busca el comentario y
    lo guarda en la columna 'comentario'
    """
    df_len = len(df)
    df['comentario'] = None
    if 'url_articulo' not in df.columns:
        raise ValueError("Error: La columna 'url_articulo' no existe en el DataFrame.")
    print("Tiempo estimado de inicio: 25s...")
    # setup del driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--enable-unsafe-webgl')
    options.add_argument('--disable-usb')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    driver.get(df.loc[0]['url_articulo']) 
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click()  # Quita cookies y carga primera web
    for index in range(len(df)):
        start_time = time.time()  # Marca de tiempo al inicio de la iteración
        print(index, '/', df_len)
        driver.get(df.loc[index]['url_articulo']) 
        try:
            elemento_comentario = driver.find_element(By.CSS_SELECTOR, "#__next > main > div > div.item-detail_ItemDetail__container__8p25r.pb-5 > section.justify-content-around.d-flex.py-4 > div.item-detail_ItemDetail__card__jnUEv.pb-5 > div.px-4 > div.item-detail_ItemDetail__separator__SCH3p.py-2 > section").text
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url_articulo'], ': \n', elemento_comentario)
        except:
            #element = driver.find_element(By.CLASS_NAME, "not-found-page_Error__title__Dky_8")
            df.loc[index, 'comentario'] = "not found"
            print(df.loc[index]['url_articulo'], df.loc[index, 'comentario'])

        end_time = time.time()  # Marca de tiempo al final de la iteración
        execution_time = end_time - start_time  # Calcula el tiempo de ejecución
        print(f"Estimated time to finish: {((df_len - index) * execution_time/60):.2f}min")
        print(f"{execution_time:.2f}s")  
    driver.quit()
    return df

In [ ]:
def get_description(n):
    """ download the description of products that doesnt satisfied the aimed condition"""
    n = str(n)
    filepath = os.path.join(carpeta_origen, f"{elemento_a_buscar.split()[0]}_{fecha}_{n}.csv")
    # Si el archivo no existe para
    if not os.path.exists(filepath):
        return 
    df_original = pd.read_csv(filepath) 
    ###############################
    # Selecciono el sub-dataframe #
    ###############################
    # Selecciono:       # as_good_as_new 
    #                   # iphone 15. 
    #                   bateria = NaN
    # estado = new...
    df_to_scrap = df_original[#(df_original['estado'] == estado) & 
                            #(df_original['gen'] == iphone) & 
                            (pd.isna(df_original['bateria']))]

    df_to_scrap.reset_index(drop=True, inplace = True)
    df_with_comments = scrap_comments(df_to_scrap)
    ##########################
   
    subdf_with_comments = df_with_comments[['id', 'comentario']]
    df_original = pd.merge(df_original, subdf_with_comments, on='id', how='left')
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    df_original.to_csv(os.path.join(carpeta_destino, f"{elemento_a_buscar}_{fecha}_{n}.csv"))
for n in range(10):
    get_description(n)

Tiempo estimado de inicio: 25s...
0 / 3
https://es.wallapop.com/item/iphone-15-1105625441 : 
 Detalles del producto
Iphone 15 de 128G con garantia hasta el 7 de Junio , usado unos meses pero en perfecto estado siempre con protectores , incluyen 4 carcasas.
Estado
Como nuevo
Marca
Apple
Modelo
iPhone 15
Color
Azul
Capacidad de almacenamiento
128 GB
Estimated time to finish: 0.40min
7.98s
1 / 3
https://es.wallapop.com/item/iphone-15-1105620514 : 
 Detalles del producto
🔍 Detalles del Producto:

- Marca: Apple
- Modelo: iPhone 15
- Almacenamiento: 128 GB
- 📱 Estado: Funcionamiento perfecto y estética increíble
- Todas las piezas del iPhone Originales ✅
- 🔋 Batería: 92% (Dura todo el día)
- ✅ Face ID: Funciona Perfectamente

🎁 Incluye:

- Caja con Manuales
- Cable de Carga y Cargador
- Funda
- Llave para tarjeta SIM

⭐ Valoraciones: 5 estrellas de clientes satisfechos. Compra con confianza.

🚀 ¡No pierdas esta oportunidad única! Contáctame ya y hazte con tu nuevo iPhone.

📍 Compra en

In [ ]:
# no existe https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182
# si existe https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441